In [1]:
# Import All Libraries
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the Boston dataset
boston = datasets.load_boston()
boston_df = pd.DataFrame(boston.data, columns=boston.feature_names)

#Splitting the dataset into predictors (X) and response variable (y)
X = boston_df.drop('CRIM', axis=1)             #Creating dataset of predictors
y = boston_df['CRIM']                          #Creating dataset of target

#Feature Scaling
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)  #Use Scaled Dataset

In [3]:
#Question 15B

X_1 = sm.add_constant(X)                         #Adding a constant column for the intercept term

#Creating a model and fitting the data
model = sm.OLS(y, X_1).fit()                     #Fitting the multiple regression (OLS) model
summary = model.summary()                        #Generating the model summary
print(summary)

print("\n" + "*" *100 )

#Creating a DataFrame of significant predictors with p-values less than 0.05
significant_predictors = pd.DataFrame(model.pvalues[model.pvalues < 0.05], columns=['p-value'])

# Displaying the significant predictors
if not significant_predictors.empty:
    display(HTML('<h3>Predictors with p < 0.05. For these predictors, we can reject the null hypothesis H0 : βj = 0</h3>'))
    display(significant_predictors)
else:
    print("No predictors with p-values less than 0.05 were found.")

                            OLS Regression Results                            
Dep. Variable:                   CRIM   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.428
Method:                 Least Squares   F-statistic:                     32.55
Date:                Wed, 19 Jul 2023   Prob (F-statistic):           4.84e-55
Time:                        22:31:22   Log-Likelihood:                -1658.8
No. Observations:                 506   AIC:                             3344.
Df Residuals:                     493   BIC:                             3398.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.9967      6.979      1.432      0.1

p-value
DIS    9.287402e-03
RAD    1.597550e-09
B      8.931725e-03
LSTAT  6.524119e-04

In [4]:
# Define the models
models = ['Regression', 'MLP']

# Define the regularization techniques for Regression
regularization_techniques_lr = {
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'Elastic_Net': ElasticNet(random_state=42)
}

# Define the regularization techniques for MLP
regularization_techniques_mlp = {
    'Ridge': MLPRegressor(hidden_layer_sizes=(100,), alpha=0.01, random_state=42),
    'Lasso': MLPRegressor(hidden_layer_sizes=(100,), alpha=0.01, activation='relu', solver='adam', early_stopping=True, random_state=42),   #Adding Activation and solver for Lasso
    'Elastic Net': ElasticNet(alpha=0.01, l1_ratio=0.5)
}

#To Combine All Coefficients of Regression
coefficients = {}

for model_name in models:
    if model_name == 'Regression':
        for regularization_name, regularization in regularization_techniques_lr.items():
            model = regularization
            # KFold (default value of cv=5) Cross Validation in Performance Metrics
            # Rounding off the values upto 2 decimal places
            rmse_scores = [round(-score, 2) for score in cross_val_score(model, X_scaled, y, scoring='neg_root_mean_squared_error')] 
            r2_scores = [round(score,2) for score in cross_val_score(model, X_scaled, y, scoring='r2')]
            print(model_name, "->", regularization_name)
            # Fit model to get coefficients
            model.fit(X_scaled, y) 
            coefficients[f"coefficients_{regularization_name}"] = pd.DataFrame({'Predictor': X_scaled.columns, 'Coefficient': model.coef_[0:]}) 
            print("Root mean squared error: ", rmse_scores)
            print("R Squared Value: ", r2_scores)
            print()
        # Merge the coefficients for Ridge, Lasso, and Elastic_Net 
        coefficients_comparison = pd.merge(coefficients.get('coefficients_Ridge'), coefficients.get('coefficients_Lasso'), on='Predictor', suffixes=['_Ridge', '_Lasso']) 
        coefficients_comparison = pd.merge(coefficients_comparison, coefficients.get('coefficients_Elastic_Net'), on='Predictor') 
        coefficients_comparison = coefficients_comparison.rename(columns={'Coefficient': 'Coefficient_Elastic_Net'})
        print(coefficients_comparison)
        print()
    else:
        for regularization_name, regularization in regularization_techniques_mlp.items():
            model = regularization
            # KFold (default value of cv=5) Cross Validation in Performance Metrics
            # Rounding off the values upto 2 decimal places
            rmse_scores = [round(-score, 2) for score in cross_val_score(model, X_scaled, y, scoring='neg_root_mean_squared_error')]
            r2_scores = [round(score,2) for score in cross_val_score(model, X_scaled, y, scoring='r2')]
            print(model_name, "->", regularization_name)
            print("Root Mean squared error: ", rmse_scores)
            print("R Squared Value: ", r2_scores)
            print()

Regression -> Ridge
Root mean squared error:  [1.91, 2.73, 1.72, 10.02, 11.3]
R Squared Value:  [-23.78, -6.73, -64.22, 0.27, 0.14]

Regression -> Lasso
Root mean squared error:  [1.3, 1.6, 1.38, 10.49, 11.41]
R Squared Value:  [-10.45, -1.64, -41.02, 0.2, 0.13]

Regression -> Elastic_Net
Root mean squared error:  [1.29, 3.01, 1.14, 10.49, 11.67]
R Squared Value:  [-10.25, -8.36, -27.67, 0.2, 0.08]

   Predictor  Coefficient_Ridge  Coefficient_Lasso  Coefficient_Elastic_Net
0         ZN           0.838635          -0.000000                 0.000000
1      INDUS          -0.486432           0.000000                 0.000000
2       CHAS          -0.330290          -0.000000                -0.000000
3        NOX          -0.785334           0.000000                 0.079620
4         RM          -0.230380          -0.000000                -0.000000
5        AGE           0.037946           0.000000                 0.030492
6        DIS          -1.477511          -0.000000               

In [5]:
#SVM
median_value = boston_df['CRIM'].median()
y_svm = (boston_df['CRIM'] >= median_value).astype(int) # 0 represents "low price" and 1 represents "high price"

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_svm, test_size=0.2, random_state=42)

#Define Scaling Function
def scale_data(train, test):
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    return train_scaled, test_scaled

#Feature Scaling
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)

#Ridge Regularisation
for model in (RidgeCV(), LassoCV(), ElasticNetCV()):
    model.fit(X_train_scaled, y_train)                           # Fitting the model
    coefficients = model.coef_                                  
    X_train_model = X_train_scaled * (coefficients != 0) 
    X_test_model = X_test_scaled * (coefficients != 0)

    # Scale the data again
    X_train_scaled1, X_test_scaled1 = scale_data(X_train_model, X_test_model)

    # Create and train the SVM model
    svm = SVC()
    svm.fit(X_train_scaled1, y_train)

    y_pred = svm.predict(X_test_scaled1)

    # Calculate accuracy
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Calculate F1 score
    f1 = metrics.f1_score(y_test, y_pred, average='weighted')

    # Calculate recall
    recall = metrics.recall_score(y_test, y_pred, average='weighted')

    # Print the scores
    print(model)
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print()

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))
Accuracy: 0.8921568627450981
F1 Score: 0.892001005046011
Recall: 0.8921568627450981

LassoCV()
Accuracy: 0.8725490196078431
F1 Score: 0.8721182340633019
Recall: 0.8725490196078431

ElasticNetCV()
Accuracy: 0.8725490196078431
F1 Score: 0.8721182340633019
Recall: 0.8725490196078431



In [6]:
# Grade G3 Predictor
g3_df = pd.read_excel(r'Grade_G3_Predictor_Transformed.xlsx')       #Edit location of Dataset Excel File

#Splitting the dataset into predictors (X) and response variable (y)
drop_col = ['G3','G3 Buckets']
X = g3_df.drop(drop_col, axis=1)         #Creating dataset of predictors            
y = g3_df['G3']                          #Creating dataset of target

#Feature Scaling
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [7]:
#Hypothesis Testing for Regression without Regularisation

X_1 = sm.add_constant(X)                         #Adding a constant column for the intercept term

#Creating a model and fitting the data
model = sm.OLS(y, X_1).fit()                     # Fitting the multiple regression (OLS) model
summary = model.summary()                        # Generating the model summary
print(summary)

print("\n" + "*" *100 )

#Creating a DataFrame of significant predictors with p-values less than 0.05
significant_predictors = pd.DataFrame(model.pvalues[model.pvalues < 0.05], columns=['p-value'])

# Displaying the significant predictors
if not significant_predictors.empty:
    display(HTML('<h3>Predictors with p < 0.05. For these predictors, we can reject the null hypothesis H0 : βj = 0</h3>'))
    display(significant_predictors)
else:
    print("No predictors with p-values less than 0.05 were found.")

                            OLS Regression Results                            
Dep. Variable:                     G3   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     10.86
Date:                Wed, 19 Jul 2023   Prob (F-statistic):           6.31e-40
Time:                        22:31:36   Log-Likelihood:                -1544.0
No. Observations:                 649   AIC:                             3150.
Df Residuals:                     618   BIC:                             3289.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.3670      1.936      3.805      0.0

p-value
const      1.561312e-04
school     2.976494e-07
sex        1.867244e-02
studytime  4.459879e-03
failures   1.210778e-11
schoolsup  4.319666e-04
higher     5.163729e-06
health     2.107224e-02

In [8]:
# Define the models
models = ['Regression', 'MLP']

# Define the regularization techniques
regularization_techniques_lr = {
    'Ridge': Ridge(random_state=42),
    'Lasso': Lasso(random_state=42),
    'Elastic_Net': ElasticNet(random_state=42)
}

regularization_techniques_mlp = {
    'Ridge': MLPRegressor(hidden_layer_sizes=(100,), alpha=0.01, random_state=42),
    'Lasso': MLPRegressor(hidden_layer_sizes=(100,), alpha=0.01, activation='relu', solver='adam', early_stopping=True, random_state=42),
    'Elastic Net': ElasticNet(alpha=0.01, l1_ratio=0.5)
}

coefficients = {}

for model_name in models:
    if model_name == 'Regression':
        for regularization_name, regularization in regularization_techniques_lr.items():
            model = regularization
            # KFold (default value of cv=5) Cross Validation in Performance Metrics
            # Rounding off the values upto 2 decimal places
            rmse_scores = [round(-score, 2) for score in cross_val_score(model, X_scaled, y, scoring='neg_root_mean_squared_error')]
            r2_scores = [round(score,2) for score in cross_val_score(model, X_scaled, y, scoring='r2')]
            print(model_name, "->", regularization_name)
            # Fit model to get coefficients
            model.fit(X_scaled, y)
            coefficients[f"coefficients_{regularization_name}"] = pd.DataFrame({'Predictor': X_scaled.columns, 'Coefficient': model.coef_[0:]}) 
            print("Root mean squared error: ", rmse_scores)
            print("R Squared Value: ", r2_scores)
            print()
        # Merge the coefficients for Ridge, Lasso, and Elastic_Net 
        coefficients_comparison = pd.merge(coefficients.get('coefficients_Ridge'), coefficients.get('coefficients_Lasso'), on='Predictor', suffixes=['_Ridge', '_Lasso']) 
        coefficients_comparison = pd.merge(coefficients_comparison, coefficients.get('coefficients_Elastic_Net'), on='Predictor') 
        coefficients_comparison = coefficients_comparison.rename(columns={'Coefficient': 'Coefficient_Elastic_Net'})
        print(coefficients_comparison)
        print()
    else:
        for regularization_name, regularization in regularization_techniques_mlp.items():
            model = regularization
            # KFold (default value of cv=5) Cross Validation in Performance Metrics
            # Rounding off the values upto 2 decimal places
            rmse_scores = [round(-score, 2) for score in cross_val_score(model, X_scaled, y, scoring='neg_root_mean_squared_error')]
            r2_scores = [round(score,2) for score in cross_val_score(model, X_scaled, y, scoring='r2')]
            print(model_name, "->", regularization_name)
            print("Root Mean squared error: ", rmse_scores)
            print("R Squared Value: ", r2_scores)
            print()

Regression -> Ridge
Root mean squared error:  [3.25, 2.68, 2.13, 2.23, 3.36]
R Squared Value:  [0.09, 0.39, 0.2, 0.23, 0.15]

Regression -> Lasso
Root mean squared error:  [3.51, 3.4, 2.26, 2.59, 4.08]
R Squared Value:  [-0.07, 0.02, 0.09, -0.03, -0.25]

Regression -> Elastic_Net
Root mean squared error:  [3.37, 3.13, 2.09, 2.44, 3.74]
R Squared Value:  [0.02, 0.17, 0.23, 0.08, -0.05]

     Predictor  Coefficient_Ridge  Coefficient_Lasso  Coefficient_Elastic_Net
0       school          -0.645944          -0.000000                -0.202804
1          sex           0.289121           0.000000                 0.000000
2          age           0.227559          -0.000000                -0.000000
3      address           0.137934           0.000000                 0.000000
4      famsize           0.144319           0.000000                 0.000000
5      Pstatus           0.030943          -0.000000                -0.000000
6         Medu           0.188887           0.000000             

In [9]:
#SVM
y_svm = g3_df['G3 Buckets']

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_svm, test_size=0.4, random_state=42)

#Define Scaling Steps
def scale_data(train, test):
    scaler = StandardScaler()
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)
    return train_scaled, test_scaled

#Feature Scaling
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)

#Ridge Regularisation
for model in (RidgeCV(), LassoCV(), ElasticNetCV()):
    model.fit(X_train_scaled, y_train)
    coefficients = model.coef_
    X_train_model = X_train_scaled * (coefficients != 0) 
    X_test_model = X_test_scaled * (coefficients != 0)

    # Scale the data again
    X_train_scaled1, X_test_scaled1 = scale_data(X_train_model, X_test_model)

    # Create and train the SVM model
    svm = SVC()
    svm.fit(X_train_scaled1, y_train)

    y_pred = svm.predict(X_test_scaled1)

    # Calculate accuracy
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Calculate F1 score
    f1 = metrics.f1_score(y_test, y_pred, average='weighted')

    # Calculate recall
    recall = metrics.recall_score(y_test, y_pred, average='weighted')

    # Print the scores
    print(model)
    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print()
    

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))
Accuracy: 0.6423076923076924
F1 Score: 0.5858228342678531
Recall: 0.6423076923076924

LassoCV()
Accuracy: 0.6384615384615384
F1 Score: 0.5893442205942206
Recall: 0.6384615384615384

ElasticNetCV()
Accuracy: 0.6346153846153846
F1 Score: 0.5820620555914674
Recall: 0.6346153846153846

